In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/p

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-03-13 01:10:36--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.26MB/s    in 0.2s    

2023-03-13 01:10:37 (6.26 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

6908554
6908145
6908145


In [6]:
# Create the customers_table DataFrame
from pyspark.sql.functions import count
customers_df = df.groupby("customer_id").agg(count("customer_id"))

customers_df.show(20)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   36702702|                 2|
|   38998964|                 8|
|   49081248|                 1|
|   28358430|                 1|
|   21911449|                 1|
|   50480727|                 2|
|    9208062|                 2|
|   42672285|                 2|
|   26554579|                 3|
|   16956038|                 2|
|    6932710|                 3|
|   33714184|                 6|
|   21302050|                12|
|   17164585|                 5|
|   51616677|                 1|
|   41049980|                 2|
|     828034|                 1|
|   45932425|                 3|
|   14337293|                 7|
|   43821790|                11|
+-----------+------------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()

products_df.show(20)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0016P392O|iMicro SP-IM942 M...|
|B00PPRYEZA|CoulVue Holder an...|
|B00VZXRRGG|Hexagon Network -...|
|B007GF9N96|Computer USB-Powe...|
|B00LCK3TEG|           Fire HD 6|
|B00KMOG5FS|Fintie 10.1" Tabl...|
|B00CFL5CRI|Cable Matters 2-P...|
|B008NP2AUC|USB USB-2, USB-3 ...|
|B00OAJ5N6I|Samsung 850 EVO -...|
|B00H02XN2I|Techno Earth Rubb...|
|B00YSC6UW8|5 Mm Thick Office...|
|B00CRXZPC6|Lenovo Ideatab S6...|
|B009SKTZFY|Bosign Tabletpill...|
|B003NZSX7G|Netgear N300 WiFi...|
|B0089JIDLC|Kingston ValueRAM...|
|B001E45XT4|Plantronics GameC...|
|B002IY4V9C|Swiss Gear Men's ...|
|B000UVRU6G|SteelSeries QcK G...|
|B00X5X3LQY|EVGA Pro SLI Brid...|
|B00L49X8E6|ASUS X551 15.6-in...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id","product_id","product_parent",to_date("review_date","yyyy-MM-dd").alias("review_date")])

review_id_df.show(20)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1001R3EZZ18B3|   13512246|B0096PD2SC|     422768802| 2013-01-27|
|R1004BM171V49W|   51335459|B004BFLP4A|     594900069| 2014-02-10|
|R100UWA60RMCZO|    7512500|B00FRHTTIU|      16015786| 2015-08-26|
|R100WQXUW2PBN3|   39247889|B001L1H0SC|     571829280| 2013-02-14|
| R100X9X33YXTK|   41228892|B00S9AMAPG|      92004405| 2015-05-15|
|R1018PWJWJL3DG|   52797442|B00LU7B8X0|     298756776| 2015-02-14|
|R101GZBULGZ95L|   15440915|B002NU5O3S|     463968341| 2013-04-17|
|R101IPFQKVUN9K|   16045822|B0040IEIII|     212966274| 2014-07-24|
|R101IU98XWDZM1|   15432781|B00425S1H8|     386765193| 2013-12-02|
|R101MVTU2LUCWC|   27493758|B00EXPNFUA|     124052875| 2014-08-10|
|R101WYR41JXHE2|   10863411|B00HJ0VSJ6|     627075706| 2014-12-10|
|R1024E7S3630L0|     584077|B00Y7MTL6G|     687620120| 2015-08

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1001R3EZZ18B3|          3|            0|          0|   N|                Y|
|R1004BM171V49W|          4|            1|          1|   N|                Y|
|R100UWA60RMCZO|          5|            0|          0|   N|                Y|
|R100WQXUW2PBN3|          5|            1|          1|   N|                Y|
| R100X9X33YXTK|          3|            0|          0|   N|                Y|
|R1018PWJWJL3DG|          4|            0|          0|   N|                Y|
|R101GZBULGZ95L|          5|            0|          0|   N|                Y|
|R101IPFQKVUN9K|          5|            0|          0|   N|                Y|
|R101IU98XWDZM1|          5|            1|          1|   N|                Y|
|R101MVTU2LUCWC|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [12]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazon-product-review.cv7yor3kz11p.us-east-2.rds.amazonaws.com:5432/amazon-product-review"
config = {"user":"postgres2", 
          "password": "Postgres2now", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)